In [10]:
from azureml.core import Workspace

ws = Workspace.from_config()  # assumes you have a config.json file

In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


compute_target = ComputeTarget(workspace=ws, name="owenb2")

In [12]:
from azureml.core import Experiment, ScriptRunConfig, Environment
experiment_name = 'rest-base-instruction-gen'
experiment = Experiment(ws, name=experiment_name)
env = Environment.get(workspace=ws, name="rest_base_gen_2", version="4")

In [18]:
from azureml.core.runconfig import DockerConfiguration
from azureml.data import OutputFileDatasetConfig

output_dir = OutputFileDatasetConfig(destination=(ws.get_default_datastore(), 'output'))
docker_config = DockerConfiguration(use_docker=True, shm_size="400g")

src = ScriptRunConfig(source_directory='.',
                      script='rest_base_instruction_gen.py',
                      arguments=['--output_dir', output_dir],
                      compute_target=compute_target,
                      environment=env,
                      docker_runtime_config=docker_config)

run = experiment.submit(config=src)
run.wait_for_completion(show_output=True)

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/owenb2/code/Users/owenb directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


RunId: rest-base-instruction-gen_1716156520_0056819f
Web View: https://ml.azure.com/runs/rest-base-instruction-gen_1716156520_0056819f?wsid=/subscriptions/76ea09d7-5d6b-45bb-b1c9-f504ad119f67/resourcegroups/beta-resources/workspaces/open-source-beta&tid=7fb1cf2d-3c32-4cd1-a802-8de4f70a9319


In [ ]:
run.register_dataset(name='shard_output', dataset=output_dir)